In [54]:
import cv2
import numpy as np
import glob
import os

In [55]:
# ▷ 폴더 경로
folder = r"C:\Users\user\Downloads\새 폴더"


In [56]:

def srgb_to_linear(x):
    x = np.clip(x,0,1); a=0.055
    return np.where(x<=0.04045, x/12.92, ((x+a)/(1+a))**2.4)

def linear_to_srgb(x):
    x = np.clip(x,0,1); a=0.055
    return np.where(x<=0.0031308, 12.92*x, (1+a)*np.power(x,1/2.4)-a)

def imread_rgba_01(path):
    d = np.fromfile(path, dtype=np.uint8)
    im = cv2.imdecode(d, cv2.IMREAD_UNCHANGED)
    if im is None: return None
    if im.ndim==2: im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGRA)
    elif im.shape[2]==3: im = cv2.cvtColor(im, cv2.COLOR_BGR2BGRA)
    return im.astype(np.float32)/255.0

exts = ("*.png","*.jpg","*.jpeg","*.bmp")
files=[]
for e in exts: files += glob.glob(os.path.join(folder,e))
files.sort()
if not files: raise SystemExit("no images")

imgs=[imread_rgba_01(f) for f in files]
imgs=[im for im in imgs if im is not None]
H,W=imgs[0].shape[:2]
imgs=[cv2.resize(im,(W,H),interpolation=cv2.INTER_AREA) for im in imgs]
N=len(imgs)

premul = [np.dstack([im[...,:3]*im[...,3:4], im[...,3:4]]) for im in imgs]
bg_p = np.median(np.stack([p[...,:3] for p in premul],axis=0),axis=0)
bg_a = np.median(np.stack([p[...,3] for p in premul],axis=0),axis=0)
bg_rgba = np.dstack([np.where(bg_a[...,None]>1e-8, bg_p/bg_a[...,None], 0.0), np.clip(bg_a,0,1)])
bg_rgb = bg_rgba[...,:3]

blur_ksize=7; diff_gain=2.0; use_otsu=True; fixed_thresh=18; dilate_iter=0; erode_iter=2
masks=[]
for i in range(N):
    src_p = premul[i][...,:3]
    diff = np.abs(cv2.cvtColor(src_p, cv2.COLOR_BGR2GRAY) - cv2.cvtColor(bg_p, cv2.COLOR_BGR2GRAY))
    diff = cv2.GaussianBlur(diff,(blur_ksize,blur_ksize),0)
    diff = np.clip(diff*diff_gain,0,1)
    if use_otsu:
        _, m = cv2.threshold((diff*255).astype(np.uint8),0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    else:
        _, m = cv2.threshold((diff*255).astype(np.uint8),fixed_thresh,255,cv2.THRESH_BINARY)
    k=np.ones((3,3),np.uint8)
    if erode_iter>0: m=cv2.erode(m,k,iterations=erode_iter)
    if dilate_iter>0: m=cv2.dilate(m,k,iterations=dilate_iter)
    masks.append((m.astype(np.float32)/255.0)[...,None])

weights = [(i+1)/N for i in range(N)]

rgbs_lin = [srgb_to_linear(im[...,:3]) for im in imgs]
alphas   = [im[...,3:4] for im in imgs]

num = np.zeros((H,W,3),np.float32)
den = np.zeros((H,W,1),np.float32)
one_minus = np.ones((H,W,1),np.float32)

for rgb,a,w,m in zip(rgbs_lin, alphas, weights, masks):
    wm = (w*m).astype(np.float32)
    num += rgb*a*wm
    den += a*wm
    one_minus *= (1.0 - a*wm)

trail_rgb_lin = np.where(den>1e-8, num/den, 0.0)
trail_alpha   = 1.0 - one_minus
trail_rgba = np.dstack([np.clip(trail_rgb_lin,0,1), np.clip(trail_alpha,0,1)])

bg_lin = srgb_to_linear(bg_rgb)
final_rgb_lin = trail_rgba[...,:3] + bg_lin*(1.0 - trail_rgba[...,3:4])
final_rgb = linear_to_srgb(final_rgb_lin)
out = np.dstack([np.clip(final_rgb,0,1), np.ones_like(trail_rgba[...,3:4])])

# ─────────────────────────────────────────────────────────
# 저장
save_path = os.path.join(folder, f"motion_composite_final_v4_{N}.png")
# ... (저장 로직은 동일)
ok, buf = cv2.imencode(".png", (out*255).astype(np.uint8)) 
if ok:
    buf.tofile(save_path)
    print(f"✅ 저장 완료: {save_path}")
else:
    print("⚠️ 저장 실패")

C:\Users\user\AppData\Local\Temp\ipykernel_27040\2495151561.py:32: RuntimeWarning: invalid value encountered in divide
  bg_rgba = np.dstack([np.where(bg_a[...,None]>1e-8, bg_p/bg_a[...,None], 0.0), np.clip(bg_a,0,1)])
C:\Users\user\AppData\Local\Temp\ipykernel_27040\2495151561.py:66: RuntimeWarning: invalid value encountered in divide
  trail_rgb_lin = np.where(den>1e-8, num/den, 0.0)


✅ 저장 완료: C:\Users\user\Downloads\새 폴더\motion_composite_final_v4_5.png
